# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [103]:
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace,split
from composable import from_toolz as tlz
from composable import pipeable
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
import re

In [2]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI')
yelp_restaurant = BeautifulSoup(r.content, "html.parser")

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [131]:

restaurant_name = (yelp_restaurant
>>find_all('a')
                  )
restaurant_name

[<a class="link__09f24__1kwXV logo-link__09f24__3oxr_ link-color--blue-dark__09f24__2DRa0 link-size--default__09f24__3xWLF" href="/" name="" rel="" role="link" target="">Yelp</a>,
 <a class="button__09f24__1hAjX secondary__09f24__3mm8J find-near-button__09f24__UO8nW" data-testid="responsive-suggest-submit" href="#" style="--mousedown-x:0px;--mousedown-y:0px;--button-width:0px"><div class="button-content__09f24__2SF6G border-color--default__09f24__R1nRO"><span class="text__09f24__2tZKC button-content-text__09f24__3OonI text-color--inherit__09f24__1jgBv text-align--center__09f24__3NO89 text-weight--semibold__09f24__MTlNc text-size--large__09f24__3-9KJ text--truncated__09f24__2NCmr">Cancel</span></div></a>,
 <a class="header-link__09f24__3OkYO" href="/advertise/consumer_header_redirect"><div class="padding-t1__09f24__2GVpG padding-r1__09f24__23Vay padding-b1__09f24__1d8yO padding-l1__09f24__aqYTU border-color--default__09f24__R1nRO"><div class="notification-wrapper__09f24__3YKGg display--

In [4]:

not_empty = re.compile(r'.+')
restaurant_name = (yelp_restaurant
>>find_all('a',attrs = {'name': not_empty})
>> map(tlz.get('name'))
)
restaurant_name

['Papa Murphy’s',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'Howie’s on La Crosse',
 'The Charmant',
 'The Freighthouse Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher',
 'Piggy’s Restaurant']

In [45]:
len(restaurant_name)

11

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [ ]:
is_ranking = re.compile(r'black-regular')
restaurant_ranking = (yelp_restaurant
>>find_all('span', attrs = {'class':is_ranking})
>>map(get_text)
)
restaurant_ranking

In [ ]:
rank_filter = re.compile(r'\d{1,2}')
rankings = [line for line in restaurant_ranking if rank_filter.match(line)]
rankings

In [ ]:
[line.replace('\xa0',"") for line in rankings]

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [ ]:
not_empty = re.compile(r'.+')
restaurant_name = (yelp_restaurant
>>find_all('a',attrs = {'name': not_empty})
>> map(tlz.get('name'))
)
restaurant_name

#### Rating

In [ ]:
star_rating = re.compile(r'\d+ star rating')
review_ranking = (yelp_restaurant
>>find_all('div', attrs = {'aria-label': star_rating})
>>map(tlz.get('aria-label'))

)
review_ranking

#### Address

In [ ]:
find_address = re.compile(r'raw_')

restaurant_address_search = (yelp_restaurant
>>find_all('span', attrs = {'class': find_address})
>>map(get_text)
)
restaurant_address_search

In [ ]:

address_filter = re.compile(r'\d{1,4}.*')
restaurant_address=[line for line in restaurant_address_search if address_filter.match(line)]
restaurant_address


#### Review Count

In [ ]:
find_reveiw_count = re.compile(r'reviewCount_')
restaurant_reveiw_count = (yelp_restaurant
>>find_all('span', attrs = {'class': find_reveiw_count})
>>map(get_text)
)
restaurant_reveiw_count

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [ ]:
restaurant_category = (yelp_restaurant
>>find_all('div')

)
restaurant_category

In [310]:
find_category = re.compile(r'priceCategory')
restaurant_category = (yelp_restaurant
>>find_all('div', attrs = {'class': find_category})
>>map(get_text)                  
)
restaurant_category

[]

In [ ]:
category_filter = [line.replace('$',"") for line in restaurant_category]
category_filter

In [ ]:
category_semi = [line.replace(',',";") for line in category_filter]
category_semi

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [ ]:
def webscraping_yelp(website):
    
    import requests
    from bs4 import BeautifulSoup
    
    s = requests.Session()
    r = s.get(website)
    yelp_restaurant = BeautifulSoup(r.content, "html.parser")
    
    
    is_ranking = re.compile(r'black-regular')
    restaurant_ranking = (yelp_restaurant
    >>find_all('span', attrs = {'class':is_ranking})
    >>map(get_text)
    )
    
    rank_filter = re.compile(r'\d{1,2}')
    rankings = [line for line in restaurant_ranking if rank_filter.match(line)]
    final_rank_name = [line.replace('\xa0'," ") for line in rankings]
    
    
    
    star_rating = re.compile(r'\d+ star rating')
    review_ranking = (yelp_restaurant
    >>find_all('div', attrs = {'aria-label': star_rating})
    >>map(tlz.get('aria-label'))
    )
    review_ranking= review_ranking[1:]
    
    find_address = re.compile(r'raw_')
    restaurant_address_search = (yelp_restaurant
    >>find_all('span', attrs = {'class': find_address})
    >>map(get_text)
    )
    address_filter = re.compile(r'\d{1,4}.*')
    restaurant_address=[line for line in restaurant_address_search if address_filter.match(line)]
    restaurant_address = restaurant_address[1:]
    
    find_reveiw_count = re.compile(r'reviewCount_')
    restaurant_reveiw_count = (yelp_restaurant
    >>find_all('span', attrs = {'class': find_reveiw_count})
    >>map(get_text)
    )
    restaurant_reveiw_count=restaurant_reveiw_count[1:]
    
    find_category = re.compile(r'priceCategory')
    restaurant_category = (yelp_restaurant
    >>find_all('div', attrs = {'class': find_category})
    >>map(get_text)                  
    )
    restaurant_category = restaurant_category[1:]
    category_filter = [line.replace('$',"") for line in restaurant_category]
    category_semi = [line.replace(',',";") for line in category_filter]
    
    joinlines = [",".join(lines) for lines in page_one]
    
    output = list(zip(final_rank_name,review_ranking,restaurant_address,restaurant_reveiw_count,category_semi))
    joinlines = [",".join(lines) for lines in output]
    joinlines_work = "\n".join(joinlines)
    return(joinlines_work)

In [ ]:
page_one = webscraping_yelp('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI%2C%20US&start=0')
page_one

In [ ]:
joinlines = [",".join(lines) for lines in page_one]
joinlines

In [ ]:
joinlines_work = "\n".join(joinlines)
joinlines_work

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

In [ ]:
webpage_number = re.compile(r'pagination-link')
page_num = (yelp_restaurant
>>find_all('a', attrs = {'class': webpage_number})
>>map(tlz.get('href'))
    )
page_num

In [ ]:
add_http = [link.replace("/","https://wwww.yelp.com/") for link in page_num]
add_http

In [ ]:
restof_pages = [webscraping_yelp(pages) for pages in add_http]
restof_pages

In [ ]:
restof_pages_work = '\n'.join(restof_pages)
restof_pages_work

In [ ]:
csv_content_one = joinlines_work

In [ ]:
with open('Yelp_LaCrosse_Restaurants_final.csv','w') as outfile:
    outfile.write(csv_content_one)

In [ ]:
with open('Yelp_LaCrosse_Restaurants_final.csv','a') as outfile:
    outfile.write(restof_pages_work)

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.